# Centroid-based samplinig of data

Here we same the the dataset that was stitched toghether in the _greg-features_ notebook. We need to do this otherwise some drugs would be overrepresented in the training data and therefore any models are more likely to overfit accordingly.

Our approach is to more fairly represent each drug. For each cluster (or vertical) of PIDs (proteins) associated with a CID (drug/molecule), we find the centroid (mean) and then find the entry closest to that. We also then find the furthest entry from the centroid. We do this for acttive and inactive interactions and therefore have a maximum of four data points for any given CID (two for active and two for inactive). By taking the centroid we aim to take an approximately _typical_ exemplar PID for the CID; by also taking the furthest from the exemplar we aim to introduce some differentiating protein characteristics to help train the model.

<hr>
We assume the data (**features.h5**, the full feature set) is in a sub-directory of the **/data** folder. I've already added entries to the _.gitignore_ file so that they won't be committed to the repository. Note that this file should be updated for new versions of the data.

See the [data readme in the Gitbug repository](https://github.com/BrianDavisMath/FDA-COVID19/tree/master/data) for more details.

<hr>

In [3]:
%pylab inline
%autosave 25

import h5py
import random
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


Autosaving every 25 seconds


## Data location

Change this when you get a new data set.

In [4]:
data_loc = '../data/FDA-COVID19_files_v1.0/'

## Load the data from features.h5
<hr>

In [7]:
store = pd.HDFStore(data_loc + 'features.h5')
df_features = pd.DataFrame(store['df' ])
print('rows: {:,}, columns: {:,}'.format(len(df_features), len(df_features.columns)))
store.close()

rows: 184,063, columns: 17,076


## Group

#### Group by cid and activity so we can find the centroid and furthest from the centroid in each group

In [8]:
df_grouped = df_features.groupby(['cid', 'activity'])

# Look at the group ("vertical") for cid 204 with active interactions.
df_grouped.get_group((204, 1))

,cid,pid,activity,AEK,VEL,EKF,LGM,VKN,LKP,NEE,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
41,204,CAA96025,1,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.0,-1.000000,...,0,0,0,0,0,0,0,0,0,0
43,204,XP_717710,1,2.749244,-1.000000,-1.0,-1.0,2.904278,-1.0,-1.000000,...,0,0,0,0,0,0,0,0,0,0
44,204,O86157,1,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.0,-1.000000,...,0,0,0,0,0,0,0,0,0,0
52,204,AAC57158,1,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.0,-1.000000,...,0,0,0,0,0,0,0,0,0,0
54,204,YP_001331689,1,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.0,-1.000000,...,0,0,0,0,0,0,0,0,0,0
58,204,Q99VQ4,1,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.0,-1.000000,...,0,0,0,0,0,0,0,0,0,0
60,204,P57771,1,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.0,-1.000000,...,0,0,0,0,0,0,0,0,0,0
62,204,P01584,1,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.0,-1.000000,...,0,0,0,0,0,0,0,0,0,0
64,204,ABY84639,1,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.0,-1.000000,...,0,0,0,0,0,0,0,0,0,0
72,204,O95372,1,0.249478,-1.000000,-1.0,-1.0,-1.000000,-1.0,-1.000000,...,0,0,0,0,0,0,0,0,0,0


## Get the representative CIDs

Note that were basing this on the full data set, including the cid vectors. 

A performance improvement would be to use data based on the merging of the interactions (cid, pid, activity) data and the protein data only and therefore reduce the dimensionality when measuring distances, i.e. not include cid features.

In [9]:
def get_representative_cid_indices(df_group):
    # remove cid, pid and activity
    df_group.drop(['cid', 'pid', 'activity'],axis=1,inplace=True)
    
    centroid = df_group.mean().values
    distances = np.sqrt((np.square(df_group.values[:,np.newaxis]-centroid).sum(axis=2)))

    idx = df_group.index
    closest_idx_to_centroid = idx[distances.argmin()]
    furthest_idx_to_centroid = idx[distances.argmax()]
    
    return [closest_idx_to_centroid, furthest_idx_to_centroid]

In [ ]:
from IPython.display import display, clear_output

num_groups = len(df_grouped.groups)
print('num groups: {:,}.'.format(num_groups))

indices = []
i = 1
for (cid, activity), group in df_grouped:
    indices.append(get_representative_cid_indices(group))
    clear_output(wait=True)
    display('processed cid group {:,} of {:,}'.format(i, num_groups))
    i = i + 1

indices = np.unique(np.array(indices).flatten())

len(indices)

'processed cid group 19,195 of 89,757'

### Save indices to file

Tos save us running the above long-running operation again.

In [ ]:
indices = np.array(indices)
h5f = h5py.File(data_loc+'indices.h5', 'w')
h5f.create_dataset('dataset_1', data=indices)
h5f.close()

# test reload from file
h5f = h5py.File(data_loc+'indices.h5','r')
b = h5f['dataset_1'][:]
h5f.close()

assert(np.allclose(indices, b)) # check our retrieved indices are the same

### Check balance of active vs inactive

In [ ]:
cids = df_features[df_features.index.isin(indices)]
grouped = cids.groupby('cid')['activity'].value_counts()
df_grouped = grouped.to_frame()
df_grouped.rename(columns={'activity': "activity_count"}, inplace=True)
df_grouped_stacked = df_grouped.unstack()
df_grouped_stacked.fillna(0, inplace=True)
df_grouped_stacked = df_grouped_stacked['activity_count']
df_grouped_stacked = df_grouped_stacked.reset_index()
df_grouped_stacked.rename_axis('', axis=1, inplace=True)

z = df_grouped_stacked[df_grouped_stacked[1] == 0]
nz = df_grouped_stacked[df_grouped_stacked[1] > 0]
print('there are {:,} drugs with zero activation'.format(len(z)))
print('there are {:,} drugs with an activation'.format(len(nz)))

### Remove indices with no active label

In [ ]:
rows_to_remove = df_grouped_stacked[df_grouped_stacked[1] == 0]['cid'].values.tolist()
del df_features
df_features = cids[~cids['cid'].isin(rows_to_remove)]
print(len(df_features))
df_features.head()

### Balance

In [ ]:
# Add some of the inactive rows back in to balance the classes.

num_to_add = len(df_features) - len(nz)
print('adding {:,} inactive rows'.format(num_to_add))

rows_to_add = random.sample(rows_to_remove, num_to_add)
df_add = cids[cids['cid'].isin(rows_to_add)]

df_features = df_features.append(df_add)

print(len(df_features))
df_features.head()

### Distribution of sampled CIDs

In [ ]:
grouped = df_features.groupby('cid')['activity'].value_counts()
df_grouped = grouped.to_frame()
df_grouped.rename(columns={'activity': "activity_count"}, inplace=True)
df_grouped_stacked = df_grouped.unstack()
df_grouped_stacked.fillna(0, inplace=True)
df_grouped_stacked = df_grouped_stacked['activity_count']
df_grouped_stacked = df_grouped_stacked.reset_index()
df_grouped_stacked.rename_axis('', axis=1, inplace=True)
df_grouped_stacked.head()

axes = df_grouped_stacked[[0, 1]].plot(kind = 'hist', subplots=True, layout = (1, 2), 
  title="frequency of cids for inactive and active interactions",
  figsize=(15, 10), xticks=[0, 1, 2], )

print('total number of cid-pid rows: {:,}'.format(len(df_features)))
print('number of features: {:,}\n'.format(len(df_features.columns)))

z = df_grouped_stacked[df_grouped_stacked[1] == 0]
nz = df_grouped_stacked[df_grouped_stacked[1] > 0]
onz = df_grouped_stacked[(df_grouped_stacked[1] > 0) & (df_grouped_stacked[0] == 0)]
print('there are {:,} drugs with zero activation'.format(len(z)))
print('there are {:,} drugs with an activation'.format(len(nz)))
print('there are {:,} drugs with only an activation\n'.format(len(onz)))

### Distribution of PIDs within sample

In [ ]:
del df_grouped_stacked
grouped = df_features.groupby('pid')['activity'].value_counts()
df_grouped = grouped.to_frame()
df_grouped.rename(columns={'activity': "activity_count"}, inplace=True)
df_grouped_stacked = df_grouped.unstack()
df_grouped_stacked.fillna(0, inplace=True)
df_grouped_stacked = df_grouped_stacked['activity_count']
df_grouped_stacked = df_grouped_stacked.reset_index()
df_grouped_stacked.rename_axis('', axis=1, inplace=True)

axes = df_grouped_stacked[[0, 1]].plot(kind = 'hist', subplots=True, layout = (1, 2), 
  title="frequency of pids for inactive and active interactions",
  figsize=(15, 10))

### Save data to file

In [ ]:
store = pd.HDFStore(data_loc + 'sampled_data.h5')
store['df'] = df_features
store.close()